<a href="https://colab.research.google.com/github/ashner-goat/Triple-Threat-Report/blob/main/Report_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report Generator

In [ ]:
# @title Input Variables

''' Google Drive Link: https://drive.google.com/drive/folders/1-9M3-Cikkq8FQEFQPBaKmh4n1JKjLIeZ '''

# To run the report cntrl + F9

# Enter Customer Name
Customer_Name = "Greg"


# Enter Seasonal Report Duration Date
start_date = "2025-01-01"
end_date = "2025-03-03"


# Enter Purchasing Options
Purchasing_Leasing_Option = "P"  # Please type only P for Purchasing or L for Leasing to avoid Error


# Enter Contracted/Outsourced Labor in USD
Outsourced_Contracted_Labor = "0"

# Enter In-House Labor Details
Base_Labor_Rate_Hour_USD = "28"
Total_Acres_Mowed = "1"
Time_Per_Mow_Per_Acre_Minutes ="35"
Times_Mowed_Per_Week = "1"
Mowing_Weeks_In_Season = "6"
Gallons_Used_Per_Acre = "1.6"
Price_Per_Gallon_USD = "4.7"
Maintenance_Cost_Per_Year_USD = "1800"
Purchase_Mowing_Gear_1time_in_5years_USD = "18000"


# Enter MyGoat Autonomous Solution Details
Number_Of_Goats_Proposed = "2"
Cost_Per_Goat_USD = "3645"
Total_Number_Of_Pens_In_Proposal = "1"
Installation_Cost_Per_Pen_USD = "1180"
Monthly_Subscription_Fee_Per_Pen_USD = "58"  # $58 if purchased and $232 if leased
Total_MyGoat_Labor_Time_Per_Ticket_Per_Pen_Per_Week_Minutes = "5"

# Enter Additional Operational Data
Unexpected_cost_incurred = "0"

# PULL ALL THE DATA IN THE POWER BI REPORT BELOW:
Goat_Activity_Ratio = {         # Just add goats if needed by following the format below
  "Cooley1465-Pen 1":"71.97",   # "Goat Name" : "Active Ratio from Power Bi Below"
  "Cooley2004-Pen 3":"77.78",

}

# PULL THE TARGET BASED ON THE
Original_KPI_PAR = "1 cut per week for 1 pen1 = 1 total cuts/wk"
MyGoat_Performance = "My Goat exceeded this KPI with an average of 2 total cuts/wk"

In [ ]:
# @title Goat Autonomy Rate
from IPython.core.display import display, HTML

power_bi_url = "https://app.powerbi.com/view?r=eyJrIjoiZjBkODZjMjctNDY4MS00YjU1LWJjODktMzczNjgzMDYzNzc0IiwidCI6IjY2NTc2OGRjLWQzYWMtNGZlMS04MzNjLTU1OTZkODY5MTBhMiIsImMiOjJ9&nav=false"

html_code = f"""
<iframe width="1200" height="680" src="{power_bi_url}" frameborder="0" allowFullScreen="true"></iframe>
"""
display(HTML(html_code))


In [ ]:
# @title Import Library
# %% Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
import plotly.graph_objects as go
from datetime import datetime, timedelta
import sqlite3
import plotly.express as px
from IPython.display import display
import re
from IPython.display import clear_output # Chan
clear_output()
import plotly.graph_objects as go

!pip install kaleido

!pip install groq
from groq import Groq

!pip install python-pptx
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping
pd.set_option('display.max_colwidth', None)  # Show full content of each column


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

db_path = '/content/drive/Shareddrives/Goat Shed/Goat Database/db_goat_ticket.db'  # Update with the correct folder name if necessary

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

Mounted at /content/drive


In [ ]:
# @title Code Verification

print("Start and End Date Validation")
start_date = datetime.strptime(start_date, "%Y-%m-%d").date()
end_date = datetime.strptime(end_date, "%Y-%m-%d").date()
print("Start and End Date Success")


print("Outsourced Contracted Labor Validation")
Outsourced_Contracted_Labor = float(Outsourced_Contracted_Labor)
print("Outsourced Contracted Labor Success")

print("In-House Labor Details Enter Validation")
base_labor_rate = float(Base_Labor_Rate_Hour_USD)
acres_mowed = float(Total_Acres_Mowed)
time_per_mow_per_acre = float(Time_Per_Mow_Per_Acre_Minutes)
mowings_per_week = float(Times_Mowed_Per_Week)
weeks_in_season = float(Mowing_Weeks_In_Season)
gallons_used_per_acre = float(Gallons_Used_Per_Acre)
price_per_gallon = float(Price_Per_Gallon_USD)
maintenance_cost = float(Maintenance_Cost_Per_Year_USD)
purchase_mowing_gear = float(Purchase_Mowing_Gear_1time_in_5years_USD)
print("In-House Labor Details Enter Validation Success")


print("Autonomous Solution Details Enter Validation")
number_of_goats = float(Number_Of_Goats_Proposed)
cost_per_goat = float(Cost_Per_Goat_USD)
number_of_pens = float(Total_Number_Of_Pens_In_Proposal)
installation_cost_per_pen = float(Installation_Cost_Per_Pen_USD)
monthly_subscription_fee_per_pen = float(Monthly_Subscription_Fee_Per_Pen_USD)
time_per_ticket = float(Total_MyGoat_Labor_Time_Per_Ticket_Per_Pen_Per_Week_Minutes)
auto_base_labor_rate = float(Base_Labor_Rate_Hour_USD)
print("Autonomous Solution Details Enter Validation Success")


Start and End Date Validation
Start and End Date Success
Outsourced Contracted Labor Validation
Outsourced Contracted Labor Success
In-House Labor Details Enter Validation
In-House Labor Details Enter Validation Success
Autonomous Solution Details Enter Validation
Autonomous Solution Details Enter Validation Success


In [ ]:
# @title Purchasing TCO Calculator Result
# Calculations

if Purchasing_Leasing_Option.upper() == "P":

  total_cuts_per_year_per_acre = mowings_per_week * weeks_in_season
  total_cuts_per_year_for_number_of_acres = total_cuts_per_year_per_acre * acres_mowed

  total_yearly_cost_of_labor = ((time_per_mow_per_acre * total_cuts_per_year_for_number_of_acres) / 60) * base_labor_rate

  total_cost_of_gas_per_year_per_acre = total_cuts_per_year_per_acre * gallons_used_per_acre * price_per_gallon
  total_cost_of_gas_per_year_for_number_of_acres = total_cost_of_gas_per_year_per_acre * acres_mowed

  total_annual_cost_in_house_mowing = (
      total_yearly_cost_of_labor +
      total_cost_of_gas_per_year_for_number_of_acres +
      maintenance_cost +
      (purchase_mowing_gear / 5)
  )

  # Display the results
  results = {
      "Total Cuts per Year per Acre": total_cuts_per_year_per_acre,
      f"Total Cuts per Year for {int(acres_mowed)} Acres": total_cuts_per_year_for_number_of_acres,
      "Total Yearly Cost of Labor (USD)": total_yearly_cost_of_labor,
      "Total Cost of Gas per Year per Acre (USD)": total_cost_of_gas_per_year_per_acre,
      f"Total Cost of Gas per Year for {int(acres_mowed)} Acres (USD)": total_cost_of_gas_per_year_for_number_of_acres,
      "Total Annual Cost for In-House Mowing (USD)": total_annual_cost_in_house_mowing
  }


  # Calculations
  total_cost_for_goats = number_of_goats * cost_per_goat
  total_installation_cost_for_number_of_pens = number_of_pens * installation_cost_per_pen
  total_monthly_subscription_fee_for_number_of_pens = number_of_pens * monthly_subscription_fee_per_pen
  total_yearly_subscription_fee_for_number_of_pens = total_monthly_subscription_fee_for_number_of_pens * 12

  # Corrected formula as per provided structure
  total_my_goat_yearly_cost_of_labor_corrected = (
      time_per_ticket * weeks_in_season * auto_base_labor_rate / 60
  ) * number_of_goats


  # Display the results
  goat_results = {
      "Total Cost for Goats (USD)": total_cost_for_goats,
      f"Total Installation Cost for {int(acres_mowed)} Pens (USD)": total_installation_cost_for_number_of_pens,
      f"Total Monthly Subscription Fee for {int(acres_mowed)} Pens (USD)": total_monthly_subscription_fee_for_number_of_pens,
      f"Total Yearly Subscription Fee for {int(acres_mowed)} Pens (USD)": total_yearly_subscription_fee_for_number_of_pens,
      "Total My Goat Yearly Cost of Labor (USD)": total_my_goat_yearly_cost_of_labor_corrected,
  }

  print("In-House Labor Details Calculation:")
  for key, value in results.items():
      print(f"{key}: {value:.2f}")

  print("-" * 80)

  print("Autonomous MyGoat Details Calculation:")
  for key, value in goat_results.items():
      print(f"{key}: {value:.2f}")




In-House Labor Details Calculation:
Total Cuts per Year per Acre: 6.00
Total Cuts per Year for 1 Acres: 6.00
Total Yearly Cost of Labor (USD): 98.00
Total Cost of Gas per Year per Acre (USD): 45.12
Total Cost of Gas per Year for 1 Acres (USD): 45.12
Total Annual Cost for In-House Mowing (USD): 5543.12
--------------------------------------------------------------------------------
Autonomous MyGoat Details Calculation:
Total Cost for Goats (USD): 7290.00
Total Installation Cost for 1 Pens (USD): 1180.00
Total Monthly Subscription Fee for 1 Pens (USD): 58.00
Total Yearly Subscription Fee for 1 Pens (USD): 696.00
Total My Goat Yearly Cost of Labor (USD): 28.00


In [ ]:
# @title Purchasing 5 Year TCO and Chart
## Business as Usual

if Purchasing_Leasing_Option.upper() == "P":

  Year_1_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor
  Year_2_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor
  Year_3_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor
  Year_4_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor
  Year_5_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor

  ## In-House Labor, Fuel and Equipment Cost
  Year_1_Labor_Costs_Per_Year = total_yearly_cost_of_labor
  Year_2_Labor_Costs_Per_Year = Year_1_Labor_Costs_Per_Year * 1.05
  Year_3_Labor_Costs_Per_Year = Year_2_Labor_Costs_Per_Year * 1.05
  Year_4_Labor_Costs_Per_Year = Year_3_Labor_Costs_Per_Year * 1.05
  Year_5_Labor_Costs_Per_Year = Year_4_Labor_Costs_Per_Year * 1.05

  Year_1_Fuel_Costs_Per_Year = total_cost_of_gas_per_year_for_number_of_acres
  Year_2_Fuel_Costs_Per_Year = Year_1_Fuel_Costs_Per_Year * 1.02
  Year_3_Fuel_Costs_Per_Year = Year_2_Fuel_Costs_Per_Year * 1.02
  Year_4_Fuel_Costs_Per_Year = Year_3_Fuel_Costs_Per_Year * 1.02
  Year_5_Fuel_Costs_Per_Year = Year_4_Fuel_Costs_Per_Year * 1.02

  Year_1_Maintenance_Cost_per_Year_Mowing_Gear = maintenance_cost
  Year_2_Maintenance_Cost_per_Year_Mowing_Gear = Year_1_Maintenance_Cost_per_Year_Mowing_Gear
  Year_3_Maintenance_Cost_per_Year_Mowing_Gear = Year_2_Maintenance_Cost_per_Year_Mowing_Gear
  Year_4_Maintenance_Cost_per_Year_Mowing_Gear = Year_3_Maintenance_Cost_per_Year_Mowing_Gear
  Year_5_Maintenance_Cost_per_Year_Mowing_Gear = Year_4_Maintenance_Cost_per_Year_Mowing_Gear

  Year_1_Onetime_Purchase_of_Mowing_Gear_within_5_Years = purchase_mowing_gear
  Year_2_Onetime_Purchase_of_Mowing_Gear_within_5_Years = 0
  Year_3_Onetime_Purchase_of_Mowing_Gear_within_5_Years = 0
  Year_4_Onetime_Purchase_of_Mowing_Gear_within_5_Years = 0
  Year_5_Onetime_Purchase_of_Mowing_Gear_within_5_Years = 0

  Year_1_Cash_Flow = Year_1_Outsourced_Contracted_Cost + Year_1_Labor_Costs_Per_Year + Year_1_Fuel_Costs_Per_Year + Year_1_Maintenance_Cost_per_Year_Mowing_Gear + Year_1_Onetime_Purchase_of_Mowing_Gear_within_5_Years
  Year_2_Cash_Flow = Year_2_Outsourced_Contracted_Cost + Year_2_Labor_Costs_Per_Year + Year_2_Fuel_Costs_Per_Year + Year_2_Maintenance_Cost_per_Year_Mowing_Gear + Year_2_Onetime_Purchase_of_Mowing_Gear_within_5_Years
  Year_3_Cash_Flow = Year_3_Outsourced_Contracted_Cost + Year_3_Labor_Costs_Per_Year + Year_3_Fuel_Costs_Per_Year + Year_3_Maintenance_Cost_per_Year_Mowing_Gear + Year_3_Onetime_Purchase_of_Mowing_Gear_within_5_Years
  Year_4_Cash_Flow = Year_4_Outsourced_Contracted_Cost + Year_4_Labor_Costs_Per_Year + Year_4_Fuel_Costs_Per_Year + Year_4_Maintenance_Cost_per_Year_Mowing_Gear + Year_4_Onetime_Purchase_of_Mowing_Gear_within_5_Years
  Year_5_Cash_Flow = Year_5_Outsourced_Contracted_Cost + Year_5_Labor_Costs_Per_Year + Year_5_Fuel_Costs_Per_Year + Year_5_Maintenance_Cost_per_Year_Mowing_Gear + Year_5_Onetime_Purchase_of_Mowing_Gear_within_5_Years

  Total_Cash_Flow = Year_1_Cash_Flow + Year_2_Cash_Flow + Year_3_Cash_Flow + Year_4_Cash_Flow + Year_5_Cash_Flow

  Year_1_Cumulative_Cash_Flow = Year_1_Cash_Flow
  Year_2_Cumulative_Cash_Flow = Year_1_Cumulative_Cash_Flow + Year_2_Cash_Flow
  Year_3_Cumulative_Cash_Flow = Year_2_Cumulative_Cash_Flow + Year_3_Cash_Flow
  Year_4_Cumulative_Cash_Flow = Year_3_Cumulative_Cash_Flow + Year_4_Cash_Flow
  Year_5_Cumulative_Cash_Flow = Year_4_Cumulative_Cash_Flow + Year_5_Cash_Flow

  Total_Cummulative_Cash_Flow = Year_5_Cumulative_Cash_Flow

  if Total_Cummulative_Cash_Flow == Total_Cash_Flow:
    print("Success - No Error in Business as Usual Computation")
  else:
    print("Failure")


  ## My Goat Solution
  Year_1_MyGoat_Installtion_Cost = total_installation_cost_for_number_of_pens
  Year_2_MyGoat_Installtion_Cost = 0
  Year_3_MyGoat_Installtion_Cost = 0
  Year_4_MyGoat_Installtion_Cost = 0
  Year_5_MyGoat_Installtion_Cost = 0

  Year_1_MyGoat_Monthly_Subscription_Fee = total_yearly_subscription_fee_for_number_of_pens
  Year_2_MyGoat_Monthly_Subscription_Fee = Year_1_MyGoat_Monthly_Subscription_Fee
  Year_3_MyGoat_Monthly_Subscription_Fee = Year_2_MyGoat_Monthly_Subscription_Fee
  Year_4_MyGoat_Monthly_Subscription_Fee = Year_3_MyGoat_Monthly_Subscription_Fee
  Year_5_MyGoat_Monthly_Subscription_Fee = Year_4_MyGoat_Monthly_Subscription_Fee

  Year_1_MyGoat_Labor_Costs_Per_Year = total_my_goat_yearly_cost_of_labor_corrected
  Year_2_MyGoat_Labor_Costs_Per_Year = Year_1_MyGoat_Labor_Costs_Per_Year
  Year_3_MyGoat_Labor_Costs_Per_Year = Year_2_MyGoat_Labor_Costs_Per_Year
  Year_4_MyGoat_Labor_Costs_Per_Year = Year_3_MyGoat_Labor_Costs_Per_Year
  Year_5_MyGoat_Labor_Costs_Per_Year = Year_4_MyGoat_Labor_Costs_Per_Year

  Year_1_MyGoat_Purchase = total_cost_for_goats
  Year_2_MyGoat_Purchase = Year_1_MyGoat_Purchase * 0.01
  Year_3_MyGoat_Purchase = Year_2_MyGoat_Purchase * 1
  Year_4_MyGoat_Purchase = Year_3_MyGoat_Purchase * 1
  Year_5_MyGoat_Purchase = Year_4_MyGoat_Purchase * 1

  Year_1_MyGoat_Cash_Flow = Year_1_MyGoat_Installtion_Cost + Year_1_MyGoat_Monthly_Subscription_Fee + Year_1_MyGoat_Labor_Costs_Per_Year + Year_1_MyGoat_Purchase
  Year_2_MyGoat_Cash_Flow = Year_2_MyGoat_Installtion_Cost + Year_2_MyGoat_Monthly_Subscription_Fee + Year_2_MyGoat_Labor_Costs_Per_Year + Year_2_MyGoat_Purchase
  Year_3_MyGoat_Cash_Flow = Year_3_MyGoat_Installtion_Cost + Year_3_MyGoat_Monthly_Subscription_Fee + Year_3_MyGoat_Labor_Costs_Per_Year + Year_3_MyGoat_Purchase
  Year_4_MyGoat_Cash_Flow = Year_4_MyGoat_Installtion_Cost + Year_4_MyGoat_Monthly_Subscription_Fee + Year_4_MyGoat_Labor_Costs_Per_Year + Year_4_MyGoat_Purchase
  Year_5_MyGoat_Cash_Flow = Year_5_MyGoat_Installtion_Cost + Year_5_MyGoat_Monthly_Subscription_Fee + Year_5_MyGoat_Labor_Costs_Per_Year + Year_5_MyGoat_Purchase

  Total_MyGoat_Cash_Flow = Year_1_MyGoat_Cash_Flow + Year_2_MyGoat_Cash_Flow + Year_3_MyGoat_Cash_Flow + Year_4_MyGoat_Cash_Flow + Year_5_MyGoat_Cash_Flow

  Year_1_MyGoat_Cumulative_Cash_Flow = Year_1_MyGoat_Cash_Flow
  Year_2_MyGoat_Cumulative_Cash_Flow = Year_1_MyGoat_Cumulative_Cash_Flow + Year_2_MyGoat_Cash_Flow
  Year_3_MyGoat_Cumulative_Cash_Flow = Year_2_MyGoat_Cumulative_Cash_Flow + Year_3_MyGoat_Cash_Flow
  Year_4_MyGoat_Cumulative_Cash_Flow = Year_3_MyGoat_Cumulative_Cash_Flow + Year_4_MyGoat_Cash_Flow
  Year_5_MyGoat_Cumulative_Cash_Flow = Year_4_MyGoat_Cumulative_Cash_Flow + Year_5_MyGoat_Cash_Flow

  Total_MyGoat_Cumulative_Cash_Flow = Year_5_MyGoat_Cumulative_Cash_Flow

  if Total_MyGoat_Cumulative_Cash_Flow == Total_MyGoat_Cash_Flow:
    print("Success - No Error in MyGoat Computation")
  else:
    print("Failure")

  Year_1_Savings = Year_1_Cash_Flow - Year_1_MyGoat_Cash_Flow  ## This is important in report
  Year_2_Savings = Year_2_Cash_Flow - Year_2_MyGoat_Cash_Flow
  Year_3_Savings = Year_3_Cash_Flow - Year_3_MyGoat_Cash_Flow
  Year_4_Savings = Year_4_Cash_Flow - Year_4_MyGoat_Cash_Flow
  Year_5_Savings = Year_5_Cash_Flow - Year_5_MyGoat_Cash_Flow

  Total_Savings = Year_1_Savings + Year_2_Savings + Year_3_Savings + Year_4_Savings + Year_5_Savings


  Year_1_Cumulative_Savings = Year_1_Savings
  Year_2_Cumulative_Savings = Year_1_Cumulative_Savings + Year_2_Savings
  Year_3_Cumulative_Savings = Year_2_Cumulative_Savings + Year_3_Savings
  Year_4_Cumulative_Savings = Year_3_Cumulative_Savings + Year_4_Savings
  Year_5_Cumulative_Savings = Year_4_Cumulative_Savings + Year_5_Savings

  Total_Cumulative_Savings = Year_5_Cumulative_Savings

  if Total_Savings == Year_5_Cumulative_Savings:
    print("Success - No Error in MyGoat Savings")
  else:
    print("Failure")

  Year_1_Labour_Savings = Year_1_Labor_Costs_Per_Year - Year_1_MyGoat_Labor_Costs_Per_Year ## This is important to be noted
  Year_2_Labour_Savings = Year_2_Labor_Costs_Per_Year - Year_2_MyGoat_Labor_Costs_Per_Year
  Year_3_Labour_Savings = Year_3_Labor_Costs_Per_Year - Year_3_MyGoat_Labor_Costs_Per_Year
  Year_4_Labour_Savings = Year_4_Labor_Costs_Per_Year - Year_4_MyGoat_Labor_Costs_Per_Year
  Year_5_Labour_Savings = Year_5_Labor_Costs_Per_Year - Year_5_MyGoat_Labor_Costs_Per_Year


  Percent_Total_Savings = round((Total_Savings / Total_Cash_Flow) * 100,2)


  # # Construct DataFrame directly from the given logic
  # Extend DataFrame to include all years up to Year 5
  df_tco_extended = pd.DataFrame({
      "Category": [
          "Outsourced/Contracted Cost",
          "Labor Costs Per Year",
          "Fuel Costs Per Year",
          "Yearly Maintenance Cost per Year (Mowing Gear)",
          "One-time Purchase of Mowing Gear within 5 Years",
          "Yearly Cash Flow",
          "Cumulative Cash Flow",
          "My Goat Installation Cost (One-time)",
          "My Goat Subscription Fee (Yearly)",
          "My Goat Labor Costs Per Year",
          "Goat Purchase (One-time)",
          "My Goat Yearly Cash Flow",
          "My Goat Cumulative Cash Flow",
          "Yearly Savings",
          "Cumulative Savings",
          "Labor Savings Per Year",
          "Total Savings",
          "Percent Savings"
      ],
      "Year 1": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${total_yearly_cost_of_labor:,.0f}",
          f"${total_cost_of_gas_per_year_for_number_of_acres:,.0f}",
          f"${maintenance_cost:,.0f}",
          f"${purchase_mowing_gear:,.0f}",
          f"${Year_1_Cash_Flow:,.0f}",
          f"${Year_1_Cumulative_Cash_Flow:,.0f}",

          f"${total_installation_cost_for_number_of_pens:,.0f}",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${total_my_goat_yearly_cost_of_labor_corrected:,.0f}",
          f"${total_cost_for_goats:,.0f}",
          f"${Year_1_MyGoat_Cash_Flow:,.0f}",
          f"${Year_1_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_1_Savings:,.0f}",
          f"${Year_1_Cumulative_Savings:,.0f}",
          f"${Year_1_Labour_Savings:,.0f}",
          "",
          ""
      ],
      "Year 2": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${Year_2_Labor_Costs_Per_Year:,.0f}",
          f"${Year_2_Fuel_Costs_Per_Year:,.0f}",
          f"${maintenance_cost:,.0f}",
          "$0",
          f"${Year_2_Cash_Flow:,.0f}",
          f"${Year_2_Cumulative_Cash_Flow:,.0f}",
          "$0",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${Year_2_MyGoat_Labor_Costs_Per_Year:,.0f}",
          f"${Year_2_MyGoat_Purchase:,.0f}",
          f"${Year_2_MyGoat_Cash_Flow:,.0f}",
          f"${Year_2_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_2_Savings:,.0f}",
          f"${Year_2_Cumulative_Savings:,.0f}",
          f"${Year_2_Labour_Savings:,.0f}",
          "",
          ""
      ],
      "Year 3": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${Year_3_Labor_Costs_Per_Year:,.0f}",
          f"${Year_3_Fuel_Costs_Per_Year:,.0f}",
          f"${maintenance_cost:,.0f}",
          "$0",
          f"${Year_3_Cash_Flow:,.0f}",
          f"${Year_3_Cumulative_Cash_Flow:,.0f}",
          "$0",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${Year_3_MyGoat_Labor_Costs_Per_Year:,.0f}",
          f"${Year_3_MyGoat_Purchase:,.0f}",
          f"${Year_3_MyGoat_Cash_Flow:,.0f}",
          f"${Year_3_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_3_Savings:,.0f}",
          f"${Year_3_Cumulative_Savings:,.0f}",
          f"${Year_3_Labour_Savings:,.0f}",
          "",
          ""
      ],
      "Year 4": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${Year_4_Labor_Costs_Per_Year:,.0f}",
          f"${Year_4_Fuel_Costs_Per_Year:,.0f}",
          f"${maintenance_cost:,.0f}",
          "$0",
          f"${Year_4_Cash_Flow:,.0f}",
          f"${Year_4_Cumulative_Cash_Flow:,.0f}",
          "$0",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${Year_4_MyGoat_Labor_Costs_Per_Year:,.0f}",
          f"${Year_4_MyGoat_Purchase:,.0f}",
          f"${Year_4_MyGoat_Cash_Flow:,.0f}",
          f"${Year_4_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_4_Savings:,.0f}",
          f"${Year_4_Cumulative_Savings:,.0f}",
          f"${Year_4_Labour_Savings:,.0f}",
          "",
          ""
      ],
      "Year 5": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${Year_5_Labor_Costs_Per_Year:,.0f}",
          f"${Year_5_Fuel_Costs_Per_Year:,.0f}",
          f"${maintenance_cost:,.0f}",
          "$0",
          f"${Year_5_Cash_Flow:,.0f}",
          f"${Year_5_Cumulative_Cash_Flow:,.0f}",
          "$0",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${Year_5_MyGoat_Labor_Costs_Per_Year:,.0f}",
          f"${Year_5_MyGoat_Purchase:,.0f}",
          f"${Year_5_MyGoat_Cash_Flow:,.0f}",
          f"${Year_5_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_5_Savings:,.0f}",
          f"${Year_5_Cumulative_Savings:,.0f}",
          f"${Year_5_Labour_Savings:,.0f}",
          f"${Total_Savings:,.0f}",
          f"{Percent_Total_Savings:.2f}%"
      ]
  })


  # Define the cumulative cash flow values for Business as Usual and MyGoat solution over 5 years
  years = ["Year 1", "Year 2", "Year 3", "Year 4", "Year 5"]

  # Assuming we have the following cumulative cash flow values from the previous computations
  business_as_usual_cumulative_cash_flow = [
      Year_1_Cumulative_Cash_Flow,
      Year_2_Cumulative_Cash_Flow,
      Year_3_Cumulative_Cash_Flow,
      Year_4_Cumulative_Cash_Flow,
      Year_5_Cumulative_Cash_Flow
  ]

  mygoat_cumulative_cash_flow = [
      Year_1_MyGoat_Cumulative_Cash_Flow,
      Year_2_MyGoat_Cumulative_Cash_Flow,
      Year_3_MyGoat_Cumulative_Cash_Flow,
      Year_4_MyGoat_Cumulative_Cash_Flow,
      Year_5_MyGoat_Cumulative_Cash_Flow
  ]

  # Create the plot
  fig_tco = go.Figure()

  # Add Business as Usual cumulative cash flow line
  fig_tco.add_trace(go.Scatter(
      x=years,
      y=business_as_usual_cumulative_cash_flow,
      mode='lines+markers',
      name='Business as Usual',
      line=dict(width=3)
  ))

  # Add MyGoat cumulative cash flow line
  fig_tco.add_trace(go.Scatter(
      x=years,
      y=mygoat_cumulative_cash_flow,
      mode='lines+markers',
      name='MyGoat Cash Flow',
      line=dict(width=3, dash='dot')
  ))

  # Update layout for better readability
  fig_tco.update_layout(
      title="5-Year Cumulative Cash Flow Comparison",
      xaxis_title="Years",
      yaxis_title="Cumulative Cash Flow (USD)",
      template="plotly_white",
      legend_title="Solutions",
      hovermode="x unified",
      width=950,
      height=350
  )

  # Display the plot
  fig_tco.show()

  print("-"*1000)
  print("Business As Usual")
  display(df_tco_extended.iloc[:7,:].reset_index(drop=True))

  print("-"*1000)
  print("My Goat Solution")
  display(df_tco_extended.iloc[7:13,:].reset_index(drop=True))

  print("-"*1000)
  print("Cost Saving")
  display(df_tco_extended.iloc[13:,:].reset_index(drop=True))


Success - No Error in Business as Usual Computation
Success - No Error in MyGoat Computation
Success - No Error in MyGoat Savings


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,Category,Year 1,Year 2,Year 3,Year 4,Year 5
0,Outsourced/Contracted Cost,$0,$0,$0,$0,$0
1,Labor Costs Per Year,$98,$103,$108,$113,$119
2,Fuel Costs Per Year,$45,$46,$47,$48,$49
3,Yearly Maintenance Cost per Year (Mowing Gear),"$1,800","$1,800","$1,800","$1,800","$1,800"
4,One-time Purchase of Mowing Gear within 5 Years,"$18,000",$0,$0,$0,$0
5,Yearly Cash Flow,"$19,943","$1,949","$1,955","$1,961","$1,968"
6,Cumulative Cash Flow,"$19,943","$21,892","$23,847","$25,808","$27,776"


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,Category,Year 1,Year 2,Year 3,Year 4,Year 5
0,My Goat Installation Cost (One-time),"$1,180",$0,$0,$0,$0
1,My Goat Subscription Fee (Yearly),$696,$696,$696,$696,$696
2,My Goat Labor Costs Per Year,$28,$28,$28,$28,$28
3,Goat Purchase (One-time),"$7,290",$73,$73,$73,$73
4,My Goat Yearly Cash Flow,"$9,194",$797,$797,$797,$797
5,My Goat Cumulative Cash Flow,"$9,194","$9,991","$10,788","$11,585","$12,382"


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,Category,Year 1,Year 2,Year 3,Year 4,Year 5
0,Yearly Savings,"$10,749","$1,152","$1,158","$1,164","$1,171"
1,Cumulative Savings,"$10,749","$11,901","$13,059","$14,224","$15,395"
2,Labor Savings Per Year,$70,$75,$80,$85,$91
3,Total Savings,,,,,"$15,395"
4,Percent Savings,,,,,55.42%


In [ ]:
# @title Leasing TCO Calculator Result
# Calculate Outputs with new input values

# Re-calculate Outputs with corrected conversions
# @title Purchasing TCO Calculator Result
# Calculations

if Purchasing_Leasing_Option.upper() == "L":

  total_cuts_per_year_per_acre = mowings_per_week * weeks_in_season
  total_cuts_per_year_for_number_of_acres = total_cuts_per_year_per_acre * acres_mowed

  total_yearly_cost_of_labor = ((time_per_mow_per_acre * total_cuts_per_year_for_number_of_acres) / 60) * base_labor_rate

  total_cost_of_gas_per_year_per_acre = total_cuts_per_year_per_acre * gallons_used_per_acre * price_per_gallon
  total_cost_of_gas_per_year_for_number_of_acres = total_cost_of_gas_per_year_per_acre * acres_mowed

  total_annual_cost_in_house_mowing = (
      total_yearly_cost_of_labor +
      total_cost_of_gas_per_year_for_number_of_acres +
      maintenance_cost +
      (purchase_mowing_gear / 5)
  )

  # Display the results
  results = {
      "Total Cuts per Year per Acre": total_cuts_per_year_per_acre,
      f"Total Cuts per Year for {int(acres_mowed)} Acres": total_cuts_per_year_for_number_of_acres,
      "Total Yearly Cost of Labor (USD)": total_yearly_cost_of_labor,
      "Total Cost of Gas per Year per Acre (USD)": total_cost_of_gas_per_year_per_acre,
      f"Total Cost of Gas per Year for {int(acres_mowed)} Acres (USD)": total_cost_of_gas_per_year_for_number_of_acres,
      "Total Annual Cost for In-House Mowing (USD)": total_annual_cost_in_house_mowing
  }


  # Calculations
  total_cost_for_goats = 0
  total_installation_cost_for_number_of_pens = number_of_goats * installation_cost_per_pen
  total_monthly_subscription_fee_for_number_of_pens = number_of_goats * monthly_subscription_fee_per_pen
  total_yearly_subscription_fee_for_number_of_pens = total_monthly_subscription_fee_for_number_of_pens * 12

  # Corrected formula as per provided structure
  total_my_goat_yearly_cost_of_labor_corrected = (
      time_per_ticket * weeks_in_season * auto_base_labor_rate / 60
  ) * number_of_goats


  # Display the results
  goat_results = {
      f"Total Installation Cost for {int(number_of_goats)} Pens (USD)": total_installation_cost_for_number_of_pens,
      f"Total Monthly Subscription Fee for {int(number_of_goats)} Pens (USD)": total_monthly_subscription_fee_for_number_of_pens,
      f"Total Yearly Subscription Fee for {int(acres_mowed)} Pens (USD)": total_yearly_subscription_fee_for_number_of_pens,
      "Total My Goat Yearly Cost of Labor (USD)": total_my_goat_yearly_cost_of_labor_corrected,
  }

  print("In-House Labor Details Calculation:")
  for key, value in results.items():
      print(f"{key}: {value:.2f}")

  print("-" * 80)

  print("Autonomous MyGoat Details Calculation:")
  for key, value in goat_results.items():
      print(f"{key}: {value:.2f}")



In [ ]:
# @title Leasing 5 Year TCO and Chart
## Business as Usual

if Purchasing_Leasing_Option.upper() == "L":

  Year_1_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor
  Year_2_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor
  Year_3_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor
  Year_4_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor
  Year_5_Outsourced_Contracted_Cost = Outsourced_Contracted_Labor

  ## In-House Labor, Fuel and Equipment Cost
  Year_1_Labor_Costs_Per_Year = total_yearly_cost_of_labor
  Year_2_Labor_Costs_Per_Year = Year_1_Labor_Costs_Per_Year * 1.05
  Year_3_Labor_Costs_Per_Year = Year_2_Labor_Costs_Per_Year * 1.05
  Year_4_Labor_Costs_Per_Year = Year_3_Labor_Costs_Per_Year * 1.05
  Year_5_Labor_Costs_Per_Year = Year_4_Labor_Costs_Per_Year * 1.05

  Year_1_Fuel_Costs_Per_Year = total_cost_of_gas_per_year_for_number_of_acres
  Year_2_Fuel_Costs_Per_Year = Year_1_Fuel_Costs_Per_Year * 1.02
  Year_3_Fuel_Costs_Per_Year = Year_2_Fuel_Costs_Per_Year * 1.02
  Year_4_Fuel_Costs_Per_Year = Year_3_Fuel_Costs_Per_Year * 1.02
  Year_5_Fuel_Costs_Per_Year = Year_4_Fuel_Costs_Per_Year * 1.02

  Year_1_Maintenance_Cost_per_Year_Mowing_Gear = maintenance_cost
  Year_2_Maintenance_Cost_per_Year_Mowing_Gear = Year_1_Maintenance_Cost_per_Year_Mowing_Gear
  Year_3_Maintenance_Cost_per_Year_Mowing_Gear = Year_2_Maintenance_Cost_per_Year_Mowing_Gear
  Year_4_Maintenance_Cost_per_Year_Mowing_Gear = Year_3_Maintenance_Cost_per_Year_Mowing_Gear
  Year_5_Maintenance_Cost_per_Year_Mowing_Gear = Year_4_Maintenance_Cost_per_Year_Mowing_Gear

  Year_1_Onetime_Purchase_of_Mowing_Gear_within_5_Years = purchase_mowing_gear
  Year_2_Onetime_Purchase_of_Mowing_Gear_within_5_Years = 0
  Year_3_Onetime_Purchase_of_Mowing_Gear_within_5_Years = 0
  Year_4_Onetime_Purchase_of_Mowing_Gear_within_5_Years = 0
  Year_5_Onetime_Purchase_of_Mowing_Gear_within_5_Years = 0

  Year_1_Cash_Flow = Year_1_Outsourced_Contracted_Cost + Year_1_Labor_Costs_Per_Year + Year_1_Fuel_Costs_Per_Year + Year_1_Maintenance_Cost_per_Year_Mowing_Gear + Year_1_Onetime_Purchase_of_Mowing_Gear_within_5_Years
  Year_2_Cash_Flow = Year_2_Outsourced_Contracted_Cost + Year_2_Labor_Costs_Per_Year + Year_2_Fuel_Costs_Per_Year + Year_2_Maintenance_Cost_per_Year_Mowing_Gear + Year_2_Onetime_Purchase_of_Mowing_Gear_within_5_Years
  Year_3_Cash_Flow = Year_3_Outsourced_Contracted_Cost + Year_3_Labor_Costs_Per_Year + Year_3_Fuel_Costs_Per_Year + Year_3_Maintenance_Cost_per_Year_Mowing_Gear + Year_3_Onetime_Purchase_of_Mowing_Gear_within_5_Years
  Year_4_Cash_Flow = Year_4_Outsourced_Contracted_Cost + Year_4_Labor_Costs_Per_Year + Year_4_Fuel_Costs_Per_Year + Year_4_Maintenance_Cost_per_Year_Mowing_Gear + Year_4_Onetime_Purchase_of_Mowing_Gear_within_5_Years
  Year_5_Cash_Flow = Year_5_Outsourced_Contracted_Cost + Year_5_Labor_Costs_Per_Year + Year_5_Fuel_Costs_Per_Year + Year_5_Maintenance_Cost_per_Year_Mowing_Gear + Year_5_Onetime_Purchase_of_Mowing_Gear_within_5_Years

  Total_Cash_Flow = Year_1_Cash_Flow + Year_2_Cash_Flow + Year_3_Cash_Flow + Year_4_Cash_Flow + Year_5_Cash_Flow

  Year_1_Cumulative_Cash_Flow = Year_1_Cash_Flow
  Year_2_Cumulative_Cash_Flow = Year_1_Cumulative_Cash_Flow + Year_2_Cash_Flow
  Year_3_Cumulative_Cash_Flow = Year_2_Cumulative_Cash_Flow + Year_3_Cash_Flow
  Year_4_Cumulative_Cash_Flow = Year_3_Cumulative_Cash_Flow + Year_4_Cash_Flow
  Year_5_Cumulative_Cash_Flow = Year_4_Cumulative_Cash_Flow + Year_5_Cash_Flow

  Total_Cummulative_Cash_Flow = Year_5_Cumulative_Cash_Flow

  if Total_Cummulative_Cash_Flow == Total_Cash_Flow:
    print("Success - No Error in Business as Usual Computation")
  else:
    print("Failure")


  ## My Goat Solution
  Year_1_MyGoat_Installtion_Cost = total_installation_cost_for_number_of_pens
  Year_2_MyGoat_Installtion_Cost = 0
  Year_3_MyGoat_Installtion_Cost = 0
  Year_4_MyGoat_Installtion_Cost = 0
  Year_5_MyGoat_Installtion_Cost = 0

  Year_1_MyGoat_Monthly_Subscription_Fee = total_yearly_subscription_fee_for_number_of_pens
  Year_2_MyGoat_Monthly_Subscription_Fee = Year_1_MyGoat_Monthly_Subscription_Fee
  Year_3_MyGoat_Monthly_Subscription_Fee = Year_2_MyGoat_Monthly_Subscription_Fee
  Year_4_MyGoat_Monthly_Subscription_Fee = Year_3_MyGoat_Monthly_Subscription_Fee
  Year_5_MyGoat_Monthly_Subscription_Fee = Year_4_MyGoat_Monthly_Subscription_Fee

  Year_1_MyGoat_Labor_Costs_Per_Year = total_my_goat_yearly_cost_of_labor_corrected
  Year_2_MyGoat_Labor_Costs_Per_Year = Year_1_MyGoat_Labor_Costs_Per_Year
  Year_3_MyGoat_Labor_Costs_Per_Year = Year_2_MyGoat_Labor_Costs_Per_Year
  Year_4_MyGoat_Labor_Costs_Per_Year = Year_3_MyGoat_Labor_Costs_Per_Year
  Year_5_MyGoat_Labor_Costs_Per_Year = Year_4_MyGoat_Labor_Costs_Per_Year

  Year_1_MyGoat_Purchase = total_cost_for_goats
  Year_2_MyGoat_Purchase = Year_1_MyGoat_Purchase * 0.01
  Year_3_MyGoat_Purchase = Year_2_MyGoat_Purchase * 1
  Year_4_MyGoat_Purchase = Year_3_MyGoat_Purchase * 1
  Year_5_MyGoat_Purchase = Year_4_MyGoat_Purchase * 1

  Year_1_MyGoat_Cash_Flow = Year_1_MyGoat_Installtion_Cost + Year_1_MyGoat_Monthly_Subscription_Fee + Year_1_MyGoat_Labor_Costs_Per_Year + Year_1_MyGoat_Purchase
  Year_2_MyGoat_Cash_Flow = Year_2_MyGoat_Installtion_Cost + Year_2_MyGoat_Monthly_Subscription_Fee + Year_2_MyGoat_Labor_Costs_Per_Year + Year_2_MyGoat_Purchase
  Year_3_MyGoat_Cash_Flow = Year_3_MyGoat_Installtion_Cost + Year_3_MyGoat_Monthly_Subscription_Fee + Year_3_MyGoat_Labor_Costs_Per_Year + Year_3_MyGoat_Purchase
  Year_4_MyGoat_Cash_Flow = Year_4_MyGoat_Installtion_Cost + Year_4_MyGoat_Monthly_Subscription_Fee + Year_4_MyGoat_Labor_Costs_Per_Year + Year_4_MyGoat_Purchase
  Year_5_MyGoat_Cash_Flow = Year_5_MyGoat_Installtion_Cost + Year_5_MyGoat_Monthly_Subscription_Fee + Year_5_MyGoat_Labor_Costs_Per_Year + Year_5_MyGoat_Purchase

  Total_MyGoat_Cash_Flow = Year_1_MyGoat_Cash_Flow + Year_2_MyGoat_Cash_Flow + Year_3_MyGoat_Cash_Flow + Year_4_MyGoat_Cash_Flow + Year_5_MyGoat_Cash_Flow

  Year_1_MyGoat_Cumulative_Cash_Flow = Year_1_MyGoat_Cash_Flow
  Year_2_MyGoat_Cumulative_Cash_Flow = Year_1_MyGoat_Cumulative_Cash_Flow + Year_2_MyGoat_Cash_Flow
  Year_3_MyGoat_Cumulative_Cash_Flow = Year_2_MyGoat_Cumulative_Cash_Flow + Year_3_MyGoat_Cash_Flow
  Year_4_MyGoat_Cumulative_Cash_Flow = Year_3_MyGoat_Cumulative_Cash_Flow + Year_4_MyGoat_Cash_Flow
  Year_5_MyGoat_Cumulative_Cash_Flow = Year_4_MyGoat_Cumulative_Cash_Flow + Year_5_MyGoat_Cash_Flow

  Total_MyGoat_Cumulative_Cash_Flow = Year_5_MyGoat_Cumulative_Cash_Flow

  if Total_MyGoat_Cumulative_Cash_Flow == Total_MyGoat_Cash_Flow:
    print("Success - No Error in MyGoat Computation")
  else:
    print("Failure")

  Year_1_Savings = Year_1_Cash_Flow - Year_1_MyGoat_Cash_Flow  ## This is important in report
  Year_2_Savings = Year_2_Cash_Flow - Year_2_MyGoat_Cash_Flow
  Year_3_Savings = Year_3_Cash_Flow - Year_3_MyGoat_Cash_Flow
  Year_4_Savings = Year_4_Cash_Flow - Year_4_MyGoat_Cash_Flow
  Year_5_Savings = Year_5_Cash_Flow - Year_5_MyGoat_Cash_Flow

  Total_Savings = Year_1_Savings + Year_2_Savings + Year_3_Savings + Year_4_Savings + Year_5_Savings


  Year_1_Cumulative_Savings = Year_1_Savings
  Year_2_Cumulative_Savings = Year_1_Cumulative_Savings + Year_2_Savings
  Year_3_Cumulative_Savings = Year_2_Cumulative_Savings + Year_3_Savings
  Year_4_Cumulative_Savings = Year_3_Cumulative_Savings + Year_4_Savings
  Year_5_Cumulative_Savings = Year_4_Cumulative_Savings + Year_5_Savings

  Total_Cumulative_Savings = Year_5_Cumulative_Savings

  if Total_Savings == Year_5_Cumulative_Savings:
    print("Success - No Error in MyGoat Savings")
  else:
    print("Failure")

  Year_1_Labour_Savings = Year_1_Labor_Costs_Per_Year - Year_1_MyGoat_Labor_Costs_Per_Year ## This is important to be noted
  Year_2_Labour_Savings = Year_2_Labor_Costs_Per_Year - Year_2_MyGoat_Labor_Costs_Per_Year
  Year_3_Labour_Savings = Year_3_Labor_Costs_Per_Year - Year_3_MyGoat_Labor_Costs_Per_Year
  Year_4_Labour_Savings = Year_4_Labor_Costs_Per_Year - Year_4_MyGoat_Labor_Costs_Per_Year
  Year_5_Labour_Savings = Year_5_Labor_Costs_Per_Year - Year_5_MyGoat_Labor_Costs_Per_Year


  Percent_Total_Savings = round((Total_Savings / Total_Cash_Flow) * 100,2)


  # # Construct DataFrame directly from the given logic
  # Extend DataFrame to include all years up to Year 5
  df_tco_extended = pd.DataFrame({
      "Category": [
          "Outsourced/Contracted Cost",
          "Labor Costs Per Year",
          "Fuel Costs Per Year",
          "Yearly Maintenance Cost per Year (Mowing Gear)",
          "One-time Purchase of Mowing Gear within 5 Years",
          "Yearly Cash Flow",
          "Cumulative Cash Flow",
          "My Goat Installation Cost (One-time)",
          "My Goat Subscription Fee (Yearly)",
          "My Goat Labor Costs Per Year",
          "Goat Purchase (One-time)",
          "My Goat Yearly Cash Flow",
          "My Goat Cumulative Cash Flow",
          "Yearly Savings",
          "Cumulative Savings",
          "Labor Savings Per Year",
          "Total Savings",
          "Percent Savings"
      ],
      "Year 1": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${total_yearly_cost_of_labor:,.0f}",
          f"${total_cost_of_gas_per_year_for_number_of_acres:,.0f}",
          f"${maintenance_cost:,.0f}",
          f"${purchase_mowing_gear:,.0f}",
          f"${Year_1_Cash_Flow:,.0f}",
          f"${Year_1_Cumulative_Cash_Flow:,.0f}",
          f"${total_installation_cost_for_number_of_pens:,.0f}",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${total_my_goat_yearly_cost_of_labor_corrected:,.0f}",
          f"${total_cost_for_goats:,.0f}",
          f"${Year_1_MyGoat_Cash_Flow:,.0f}",
          f"${Year_1_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_1_Savings:,.0f}",
          f"${Year_1_Cumulative_Savings:,.0f}",
          f"${Year_1_Labour_Savings:,.0f}",
          "",
          ""
      ],
      "Year 2": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${Year_2_Labor_Costs_Per_Year:,.0f}",
          f"${Year_2_Fuel_Costs_Per_Year:,.0f}",
          f"${maintenance_cost:,.0f}",
          "$0",
          f"${Year_2_Cash_Flow:,.0f}",
          f"${Year_2_Cumulative_Cash_Flow:,.0f}",
          "$0",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${Year_2_MyGoat_Labor_Costs_Per_Year:,.0f}",
          f"${Year_2_MyGoat_Purchase:,.0f}",
          f"${Year_2_MyGoat_Cash_Flow:,.0f}",
          f"${Year_2_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_2_Savings:,.0f}",
          f"${Year_2_Cumulative_Savings:,.0f}",
          f"${Year_2_Labour_Savings:,.0f}",
          "",
          ""
      ],
      "Year 3": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${Year_3_Labor_Costs_Per_Year:,.0f}",
          f"${Year_3_Fuel_Costs_Per_Year:,.0f}",
          f"${maintenance_cost:,.0f}",
          "$0",
          f"${Year_3_Cash_Flow:,.0f}",
          f"${Year_3_Cumulative_Cash_Flow:,.0f}",
          "$0",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${Year_3_MyGoat_Labor_Costs_Per_Year:,.0f}",
          f"${Year_3_MyGoat_Purchase:,.0f}",
          f"${Year_3_MyGoat_Cash_Flow:,.0f}",
          f"${Year_3_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_3_Savings:,.0f}",
          f"${Year_3_Cumulative_Savings:,.0f}",
          f"${Year_3_Labour_Savings:,.0f}",
          "",
          ""
      ],
      "Year 4": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${Year_4_Labor_Costs_Per_Year:,.0f}",
          f"${Year_4_Fuel_Costs_Per_Year:,.0f}",
          f"${maintenance_cost:,.0f}",
          "$0",
          f"${Year_4_Cash_Flow:,.0f}",
          f"${Year_4_Cumulative_Cash_Flow:,.0f}",
          "$0",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${Year_4_MyGoat_Labor_Costs_Per_Year:,.0f}",
          f"${Year_4_MyGoat_Purchase:,.0f}",
          f"${Year_4_MyGoat_Cash_Flow:,.0f}",
          f"${Year_4_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_4_Savings:,.0f}",
          f"${Year_4_Cumulative_Savings:,.0f}",
          f"${Year_4_Labour_Savings:,.0f}",
          "",
          ""
      ],
      "Year 5": [
          f"${Outsourced_Contracted_Labor:,.0f}",
          f"${Year_5_Labor_Costs_Per_Year:,.0f}",
          f"${Year_5_Fuel_Costs_Per_Year:,.0f}",
          f"${maintenance_cost:,.0f}",
          "$0",
          f"${Year_5_Cash_Flow:,.0f}",
          f"${Year_5_Cumulative_Cash_Flow:,.0f}",
          "$0",
          f"${total_yearly_subscription_fee_for_number_of_pens:,.0f}",
          f"${Year_5_MyGoat_Labor_Costs_Per_Year:,.0f}",
          f"${Year_5_MyGoat_Purchase:,.0f}",
          f"${Year_5_MyGoat_Cash_Flow:,.0f}",
          f"${Year_5_MyGoat_Cumulative_Cash_Flow:,.0f}",
          f"${Year_5_Savings:,.0f}",
          f"${Year_5_Cumulative_Savings:,.0f}",
          f"${Year_5_Labour_Savings:,.0f}",
          f"${Total_Savings:,.0f}",
          f"{Percent_Total_Savings:.2f}%"
      ]
  })


  # Define the cumulative cash flow values for Business as Usual and MyGoat solution over 5 years
  years = ["Year 1", "Year 2", "Year 3", "Year 4", "Year 5"]

  # Assuming we have the following cumulative cash flow values from the previous computations
  business_as_usual_cumulative_cash_flow = [
      Year_1_Cumulative_Cash_Flow,
      Year_2_Cumulative_Cash_Flow,
      Year_3_Cumulative_Cash_Flow,
      Year_4_Cumulative_Cash_Flow,
      Year_5_Cumulative_Cash_Flow
  ]

  mygoat_cumulative_cash_flow = [
      Year_1_MyGoat_Cumulative_Cash_Flow,
      Year_2_MyGoat_Cumulative_Cash_Flow,
      Year_3_MyGoat_Cumulative_Cash_Flow,
      Year_4_MyGoat_Cumulative_Cash_Flow,
      Year_5_MyGoat_Cumulative_Cash_Flow
  ]

  # Create the plot
  fig = go.Figure()

  # Add Business as Usual cumulative cash flow line
  fig.add_trace(go.Scatter(
      x=years,
      y=business_as_usual_cumulative_cash_flow,
      mode='lines+markers',
      name='Business as Usual',
      line=dict(width=3)
  ))

  # Add MyGoat cumulative cash flow line
  fig.add_trace(go.Scatter(
      x=years,
      y=mygoat_cumulative_cash_flow,
      mode='lines+markers',
      name='MyGoat Cash Flow',
      line=dict(width=3, dash='dot')
  ))

  # Update layout for better readability
  fig.update_layout(
      title="5-Year Cumulative Cash Flow Comparison",
      xaxis_title="Years",
      yaxis_title="Cumulative Cash Flow (USD)",
      template="plotly_white",
      legend_title="Solutions",
      hovermode="x unified",
      width=950,
      height=350
  )

  # Display the plot
  fig.show()

  print("-"*1000)
  print("Business As Usual")
  display(df_tco_extended.iloc[:7,:].reset_index(drop=True))

  print("-"*1000)
  print("My Goat Solution")
  display(df_tco_extended.iloc[7:13,:].reset_index(drop=True))

  print("-"*1000)
  print("Cost Saving")
  display(df_tco_extended.iloc[13:,:].reset_index(drop=True))


In [ ]:
# @title Tickets Table

query = """
    SELECT
        tbl_goat_tickets.id AS "Ticket ID",
        tbl_goat_tickets.goat_name AS "Goat Name",
        tbl_goat_tickets.customer_name AS "Customer",
        tbl_goat_tickets.description AS "Error Description",
        tbl_goat_tickets.reported_at AS "Reported At",
        tbl_goat_tickets.resolved_at AS "Resolved At",
        tbl_goat_tickets.lat AS "Latitude",
        tbl_goat_tickets.lng AS "Longitude",
        tbl_goat_tickets.created_at AS "Ticket Created",
        tbl_goat_tickets.resolver_first_name || ' ' || tbl_goat_tickets.resolver_last_name AS "Resolved By",
        (JULIANDAY(tbl_goat_tickets.resolved_at) - JULIANDAY(tbl_goat_tickets.reported_at)) * 24 AS "Duration (Hrs)"
    FROM
        tbl_goat_tickets
    WHERE
        tbl_goat_tickets.customer_name LIKE ?
        AND tbl_goat_tickets.reported_at BETWEEN ? AND ?
    ORDER BY
        tbl_goat_tickets.reported_at DESC;
"""


# Execute query with parameters
Tickets_Data = pd.read_sql(query, conn, params=(f"%{Customer_Name}%", start_date, end_date))

# Convert ID to Object
Tickets_Data['Ticket ID'] = Tickets_Data['Ticket ID'].astype(object)

# Extract error codes or retain the original description if no error code is found
Tickets_Data["Short Error"] = Tickets_Data["Error Description"].apply(
    lambda x: re.search(r"(Error code \d+)", x).group(0) if re.search(r"(Error code \d+)", x) else x
)

# Get counts of errors
error_counts = Tickets_Data["Short Error"].value_counts()
error_counts_full = Tickets_Data["Error Description"].value_counts()

# Get the full error descriptions corresponding to the short labels for tooltips
full_error_map = Tickets_Data.set_index("Short Error")["Error Description"].to_dict()

error_duration_avg = Tickets_Data.groupby("Short Error")["Duration (Hrs)"].mean()
error_duration_avg = error_duration_avg.sort_values(ascending=False)


# Create the bar chart
# fig1 = go.Figure([
#     go.Bar(
#         x=error_counts.index,
#         y=error_counts.values,
#         text=error_counts.values,
#         textposition='inside',
#         marker=dict(color='darkblue'),
#         hovertext=[full_error_map[err] for err in error_counts.index],  # Tooltip with full error name
#         hoverinfo="text"
#     )
# ])

# fig1.update_layout(
#     title="Chart 1: Error Type Distribution",
#     xaxis=dict(
#         title="Error Code",
#         tickangle=-90  # Rotate labels 90 degrees
#     ),
#     yaxis_title="Occurrences",
#     template="plotly_white",
#     showlegend=False,
#     width=950,
#     height=350
# )

fig1 = go.Figure([
    go.Bar(
        x=error_duration_avg.index,
        y=error_duration_avg.values,
        text=error_duration_avg.round(2),
        textposition='inside',
        marker=dict(color='darkblue'),
    )
])

fig1.update_layout(
    title="Chart 1: Average Resolution Time per Error Type",
    xaxis=dict(title="Error Code", tickangle=-90),
    yaxis_title="Average Duration (Hrs)",
    template="plotly_white",
    showlegend=False,
    width=950,
    height=350
)

fig1.show()

print("-"*1000)

fig2 = go.Figure()

# Chart 2: Distribution of Resolution Times (Scatter Plot)
fig2 = go.Figure([
    go.Scatter(
        x=Tickets_Data["Ticket ID"],
        y=Tickets_Data["Duration (Hrs)"],
        mode='markers',
        marker=dict(color='red', size=10)
    )
])

fig2.update_layout(
    title="Chart 2: Distribution of Resolution Times",
    xaxis_title="Ticket ID",
    yaxis_title="Duration (Hrs)",
    template="plotly_white",
    showlegend=False,
    width=950,
    height=350
)

# Display the charts
fig2.show()

print("-"*1000)


# Prepare the data
error_summary = Tickets_Data["Error Description"].value_counts().reset_index()
error_summary.columns = ["Error Code", "Count"]


average_error_data = {'Description': '"Total Average Sheperd Respons time per Ticket (Hours):"', 'Average Hours': [Tickets_Data['Duration (Hrs)'].mean()]}
average_error_df = pd.DataFrame(average_error_data)


display(error_summary)
print("-"*1000)
display(average_error_df)
print("-"*1000)
display(Tickets_Data.iloc[:,:-1])

print("-"*1000)

# Create a mapping of short error codes to their full descriptions
error_description_map = Tickets_Data.set_index("Short Error")["Error Description"].to_dict()

# Get the top 2 error types based on duration
top_2_errors = error_duration_avg.head(2).index.tolist()

# Retrieve descriptions for the top 2 error codes
top_2_descriptions = [error_description_map[error] for error in top_2_errors]

# Generate the dynamic summary text with descriptions
if len(top_2_errors) == 2:
    result_4_1 = (
        f"Chart 1: Error Type Distribution\n"
        f"“{top_2_errors[0]} ({top_2_descriptions[0]})” and "
        f"“{top_2_errors[1]} ({top_2_descriptions[1]})” dominate with the highest average resolution times, "
        f"while most other errors remain minimal."
    )
elif len(top_2_errors) == 1:
    result_4_1 = (
        f"Chart 1: Error Type Distribution\n"
        f"“{top_2_errors[0]} ({top_2_descriptions[0]})” has the highest average resolution time, "
        f"while most other errors remain minimal."
    )
else:
    result_4_1 = "Chart 1: Error Type Distribution\nData is insufficient to determine dominant error types."

# Print the summary with descriptions
print(result_4_1)


print("-"*1000)

# Calculate additional statistics for deeper insights
max_duration = Tickets_Data["Duration (Hrs)"].max()
avg_duration = Tickets_Data["Duration (Hrs)"].mean()
below_50 = (Tickets_Data["Duration (Hrs)"] <= 50).sum()
between_50_250 = ((Tickets_Data["Duration (Hrs)"] > 50) & (Tickets_Data["Duration (Hrs)"] <= 250)).sum()
above_250 = (Tickets_Data["Duration (Hrs)"] > 250).sum()

# Generate a refined dynamic summary for Chart 2
if above_250 > 0:
    result_4_2 = (
        f"Chart 2: Average Resolution Time by Goat\n"
        f"Most tickets are resolved within **50 to 250 hours**, while **{above_250} cases exceeded 250 hours**, "
        f"highlighting occasional delays in resolution. "
        f"The **longest resolution time recorded was {max_duration:.2f} hours**, compared to an average of **{avg_duration:.2f} hours**."
    )
elif between_50_250 > 0:
    result_4_2 = (
        f"Chart 2: Average Resolution Time by Goat\n"
        f"Resolution times are mostly **between 50 and 250 hours**, with minimal cases extending beyond that. "
        f"The **average resolution time is {avg_duration:.2f} hours**, indicating a relatively efficient resolution process."
    )
else:
    result_4_2 = (
        f"Chart 2: Average Resolution Time by Goat\n"
        f"All tickets were resolved in under **50 hours**, suggesting a highly efficient ticket resolution process."
    )

# Print the refined summary for Chart 2
print(result_4_2)


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,Error Code,Count
0,Error code 10 (Upside down) logged via Husqvarna API,6
1,Goat stopped,4
2,Error code 71 (Alarm! Mower lifted) logged via Husqvarna API,1
3,Error code 2 (No loop signal) logged via Husqvarna API,1
4,Error code 62 (Temporary battery problem) logged via Husqvarna API,1
5,Error code 15 (Lifted) logged via Husqvarna API,1
6,Error code 78 (Slipped - Mower has Slipped. Situation not solved with moving pattern) logged via Husqvarna API,1
7,"Error code 18 (Collision sensor problem, rear) logged via Husqvarna API",1
8,Error code 1 (Outside working area) logged via Husqvarna API,1


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,Description,Average Hours
0,"""Total Average Sheperd Respons time per Ticket (Hours):""",24.859624


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,Ticket ID,Goat Name,Customer,Error Description,Reported At,Resolved At,Latitude,Longitude,Ticket Created,Resolved By,Duration (Hrs)
0,26990,Cooley1465-Pen 1,Greg Cooley Inc,Error code 71 (Alarm! Mower lifted) logged via Husqvarna API,2025-03-02 22:32:59,2025-03-03 07:00:05,35.8704,-86.6745,2025-03-02 22:45:05,None,8.451667
1,26987,Cooley2004-Pen 3,Greg Cooley Inc,Error code 2 (No loop signal) logged via Husqvarna API,2025-03-01 12:37:52,2025-03-01 13:15:06,35.8704,-86.6744,2025-03-01 12:45:05,None,0.620556
2,26981,Cooley1465-Pen 1,Greg Cooley Inc,Goat stopped,2025-02-24 11:40:16,2025-02-24 13:00:05,36.1699,-86.8252,2025-02-24 11:45:06,None,1.330278
3,26980,Cooley2004-Pen 3,Greg Cooley Inc,Error code 10 (Upside down) logged via Husqvarna API,2025-02-22 06:10:13,2025-02-24 12:30:07,35.8705,-86.6746,2025-02-22 06:15:06,None,54.331667
4,26978,Cooley1465-Pen 1,Greg Cooley Inc,Error code 10 (Upside down) logged via Husqvarna API,2025-02-21 18:12:28,2025-02-24 11:15:06,35.8704,-86.5046,2025-02-21 18:15:06,None,65.043889
5,26975,Cooley2004-Pen 3,Greg Cooley Inc,Goat stopped,2025-02-19 18:00:45,2025-02-22 06:15:05,35.8704,-86.6746,2025-02-19 18:15:05,None,60.238889
6,26974,Cooley2004-Pen 3,Greg Cooley Inc,Error code 10 (Upside down) logged via Husqvarna API,2025-02-18 18:14:53,2025-02-19 17:45:05,35.9652,-86.8081,2025-02-18 18:15:05,None,23.503333
7,26973,Cooley2004-Pen 3,Greg Cooley Inc,Error code 10 (Upside down) logged via Husqvarna API,2025-02-18 16:28:23,2025-02-18 18:00:05,36.1667,-86.7782,2025-02-18 16:30:04,None,1.528333
8,26972,Cooley2004-Pen 3,Greg Cooley Inc,Goat stopped,2025-02-17 16:25:18,2025-02-18 16:30:04,35.8705,-86.6746,2025-02-17 16:30:05,None,24.079444
9,26971,Cooley1465-Pen 1,Greg Cooley Inc,Goat stopped,2025-02-17 16:10:26,2025-02-21 18:15:06,35.8705,-86.6745,2025-02-17 16:15:05,None,98.077778


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# @title Goat Autonomy Rate & Property Appearance Rating

# Creating the dataframe while preserving the original format
df_goat_ar = pd.DataFrame(list(Goat_Activity_Ratio.items()), columns=["Goat Name", "Active Ratio"])

df_goat_ar["Active Ratio"] = pd.to_numeric(df_goat_ar["Active Ratio"])

display(df_goat_ar)

print("-"*1000)

# Creating a dataframe using the provided Goat Activity Ratio data
df_plot = pd.DataFrame(list(Goat_Activity_Ratio.items()), columns=["Goat Name", "Active Ratio"])
df_plot["Active Ratio"] = df_plot["Active Ratio"].astype(float)  # Convert to numeric values

# Calculate overall property autonomy rate
overall_AR = df_plot["Active Ratio"].mean()

fig_ar = go.Figure()

# Create the bar chart using Plotly
fig_ar = px.bar(df_plot, x="Goat Name", y="Active Ratio", text="Active Ratio",
             labels={"Active Ratio": "% Autonomy Rate"}, title=f"Autonomy Rate(%) Overall Property AR of {df_goat_ar['Active Ratio'].mean():.2f}%")

# Add the overall AR as an annotation
fig_ar.add_annotation(
    x=0, y=max(df_plot["Active Ratio"]) + 5,
    text=f"Overall Property AR of {overall_AR:.2f}%",
    showarrow=False,
    font=dict(size=14, color="black"),
    bgcolor="lime",
    bordercolor="black"
)

# Display the chart
fig_ar.show()

df_ar = pd.DataFrame({
    "Original KPI": [Original_KPI_PAR],
    "My Goat Performance": [MyGoat_Performance]
})

print("-"*1000)

display(df_ar)

print("-"*1000)

df_goat_ar['Active Ratio'].max()

df_goat_ar["Active Ratio"] = pd.to_numeric(df_goat_ar["Active Ratio"])


max_active_goat = df_goat_ar.loc[df_goat_ar["Active Ratio"].idxmax(), "Goat Name"]
min_active_goat = df_goat_ar.loc[df_goat_ar["Active Ratio"].idxmin(), "Goat Name"]


# If there are more than 3 goats, get the top 2 highest Active Ratios
if len(df_goat_ar) > 2:
    top_active_goats = df_goat_ar.nlargest(2, "Active Ratio")["Goat Name"].tolist()
    result_3_1 = f""" {top_active_goats[0]} and {top_active_goats[1]} show the highest autonomy rates ({df_goat_ar.loc[df_goat_ar['Goat Name'].str.contains(top_active_goats[0])]['Active Ratio'][0]}% and {df_goat_ar.loc[df_goat_ar['Goat Name'].str.contains(top_active_goats[0])]['Active Ratio'][0]}%), while {min_active_goat} has the lowest at ~{df_goat_ar['Active Ratio'].min()}%."""

else:
    top_active_goats = max_active_goat
    result_3_1 = f""" {max_active_goat} show the highest autonomy rate ({df_goat_ar['Active Ratio'].max()}%), while {min_active_goat} has the lowest at ~{df_goat_ar['Active Ratio'].min()}%."""


display(result_3_1)



,Goat Name,Active Ratio
0,Cooley1465-Pen 1,71.97
1,Cooley2004-Pen 3,77.78


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,Original KPI,My Goat Performance
0,1 cut per week for 1 pen1 = 1 total cuts/wk,My Goat exceeded this KPI with an average of 2 total cuts/wk


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

' Cooley2004-Pen 3 show the highest autonomy rate (77.78%), while Cooley1465-Pen 1 has the lowest at ~71.97%.'

In [ ]:
# @title The My Goat Triple Threat
report_1 = f"""
Cost Savings:
Old way... {Tickets_Data['Customer'][0]} would have spent a total of ${Year_1_Cash_Flow:,.2f} on Labor (${total_yearly_cost_of_labor:,.2f}), Gas (${total_cost_of_gas_per_year_for_number_of_acres:,.2f}), and Equipment Purchases & Maintenance (${Purchase_Mowing_Gear_1time_in_5years_USD}/once every 5 years + ${Maintenance_Cost_Per_Year_USD}/year ) on their vegetation management in Year 1.
With the My Goat Solution, {Tickets_Data['Customer'].head(1)[0]} spent a total of ${Year_1_MyGoat_Cash_Flow:,.2f} on Installation ($4,720, one-time cost), Monthly SW Subscription (${total_yearly_subscription_fee_for_number_of_pens:,.2f}), Labor (${total_my_goat_yearly_cost_of_labor_corrected:,.2f}), Gas ($ ) and Goat Purchases (${total_cost_for_goats:,.2f} one-time) in Year 1 and only ${Year_2_MyGoat_Cash_Flow:,.2f} in each season following.
That’s a total SAVINGS of ${Year_1_Savings:,.2f} in your first year alone with My Goat! Over 5 years, total savings climb to ${Total_Savings:,.2f}, representing a {Percent_Total_Savings:,.2f}% reduction in Overall Costs with the My Goat Solution.
"""

Year_Total_Labor_Costs_Per_Year = Year_1_Labor_Costs_Per_Year + Year_2_Labor_Costs_Per_Year + Year_3_Labor_Costs_Per_Year + Year_4_Labor_Costs_Per_Year + Year_5_Labor_Costs_Per_Year
Year_Total_MyGoat_Labor_Costs_Per_Year = Year_1_MyGoat_Labor_Costs_Per_Year + Year_2_MyGoat_Labor_Costs_Per_Year + Year_3_MyGoat_Labor_Costs_Per_Year + Year_4_MyGoat_Labor_Costs_Per_Year + Year_5_MyGoat_Labor_Costs_Per_Year

report_1_2 = f"""
Labor Multiplier:
Old way… In one season, {Tickets_Data['Customer'][0]} would have spent ${Year_1_Cash_Flow:,.2f} on Labor expenses at ${base_labor_rate:,.2f}/hr for 1 hour per acre, per week.
With the My Goat Solution, {Tickets_Data['Customer'].head(1)[0]} has spent only ${Year_2_MyGoat_Labor_Costs_Per_Year:,.2f} on Labor expenses at only 5 minutes per acre, per week.
That is a total SAVINGS of ${Year_1_Labour_Savings:,.2f} in your first season alone with My Goat!  Over the course of 5 years, your total labor savings will be ${Year_Total_Labor_Costs_Per_Year-Year_Total_MyGoat_Labor_Costs_Per_Year:,.2f}.
That's a %{((Year_Total_Labor_Costs_Per_Year- Year_Total_MyGoat_Labor_Costs_Per_Year)/Year_Total_Labor_Costs_Per_Year)*100:,.2f} reduction in Labor Costs alone.
"""

Total_Gallons_Saved = acres_mowed * mowings_per_week * gallons_used_per_acre * round((end_date - start_date).days/7,0)
co2_per_gallon_kg = 8.887  # CO2 emissions per gallon in kg (EPA standard)
kg_to_metric_ton = 1000  # 1 metric ton = 1000 kg

co2_saved_metric_tons = (Total_Gallons_Saved * co2_per_gallon_kg) / kg_to_metric_ton

report_1_3 = f"""
Sustainability/Safety Impact:
Old way… {Tickets_Data['Customer'].head(1)[0]} would have used {gallons_used_per_acre} gallons of gas per acre mowed.
With the My Goat Solution, {Tickets_Data['Customer'].head(1)[0]} has used 0 gallons of gas mowing over {int(acres_mowed)} acres an average of {int(acres_mowed)} times a week for {int(round((end_date - start_date).days/7,0))} weeks.
This has SAVED you {Total_Gallons_Saved:,.2f} gallons of gas this season alone!  This is equivalent to {round(co2_saved_metric_tons)} metric tons of carbon emissions saved.
"""

print(report_1)
print(report_1_2)
print(report_1_3)


Cost Savings:
Old way... Greg Cooley Inc would have spent a total of $19,943.12 on Labor ($98.00), Gas ($45.12), and Equipment Purchases & Maintenance ($18000/once every 5 years + $1800/year ) on their vegetation management in Year 1.
With the My Goat Solution, Greg Cooley Inc spent a total of $9,194.00 on Installation ($4,720, one-time cost), Monthly SW Subscription ($696.00), Labor ($28.00), Gas ($ ) and Goat Purchases ($7,290.00 one-time) in Year 1 and only $796.90 in each season following.
That’s a total SAVINGS of $10,749.12 in your first year alone with My Goat! Over 5 years, total savings climb to $15,394.72, representing a 55.42% reduction in Overall Costs with the My Goat Solution.


Labor Multiplier:
Old way… In one season, Greg Cooley Inc would have spent $19,943.12 on Labor expenses at $28.00/hr for 1 hour per acre, per week.
With the My Goat Solution, Greg Cooley Inc has spent only $28.00 on Labor expenses at only 5 minutes per acre, per week.
That is a total SAVINGS of $

In [ ]:
# @title Total Cost of Ownership
fig_tco.show()

result_2_1 = f'''

TCO Chart & Interpretation:
The "My Goat" solution consistently achieves a lower TCO, resulting in a ${Total_Savings:,.2f} cost-savings compared to "Business As Usual," by Year 5.

'''

print(result_2_1)



TCO Chart & Interpretation:
The "My Goat" solution consistently achieves a lower TCO, resulting in a $15,394.72 cost-savings compared to "Business As Usual," by Year 5.




In [ ]:
# @title Next Steps & Feedback

def groq_access(content):
    try:
        client = Groq(api_key="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")
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="llama3-8b-8192",
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


feedback_result = groq_access("Give me a feedback, summary, and what are the next steps about the result:"+report_1 + report_1_2 + report_1_3 + result_2_1 + result_3_1 + result_4_1 + result_4_2)

print(feedback_result)

**Feedback:** 

The My Goat Solution has significantly improved the cost savings, labor efficiency, and sustainability of Greg Cooley Inc's vegetation management. The solution has resulted in a total cost savings of $15,394.72 over five years, a reduction in labor costs of 74.15%, and a substantial reduction in carbon emissions.

**Summary:** 

The My Goat Solution has:

* Reduced overall costs by 55.42% over five years
* Achieved a labor multiplier of 74.15%, resulting in reduced labor expenses
* Eliminated the need for gas-powered equipment, reducing carbon emissions by 14.4 gallons in the first season alone

**Next Steps:** 

1. Continue to monitor and optimize the My Goat Solution to ensure maximum efficiency and effectiveness.
2. Expand the solution to other areas of the business to further reduce costs and improve sustainability.
3. Consider incorporating additional features or upgrades to the solution to further improve labor efficiency and reduce costs.
4. Continue to track and

In [ ]:
# @title Power Point Report Generator

chart_path = "/content/drive/Shareddrives/Goat Shed/Goat Database/tco_chart.png"
fig_tco.write_image(chart_path, engine="kaleido")


chart_path_2 = "/content/drive/Shareddrives/Goat Shed/Goat Database/autonomy_chart.png"
fig_ar.write_image(chart_path_2, engine="kaleido")


chart_path_3 = "/content/drive/Shareddrives/Goat Shed/Goat Database/error_desc_1.png"
fig1.write_image(chart_path_3, engine="kaleido")

chart_path_4 = "/content/drive/Shareddrives/Goat Shed/Goat Database/resolution_chart.png"
fig2.write_image(chart_path_4, engine="kaleido")

# Create PowerPoint Presentation
prs = Presentation()

# Add Title Slide
title_slide_layout = prs.slide_layouts[5]  # Title Only Layout
title_slide = prs.slides.add_slide(title_slide_layout)
title = title_slide.shapes.title
title.text = Tickets_Data['Customer'].head(1)[0]
title.text_frame.paragraphs[0].runs[0].font.bold = True
title.text_frame.paragraphs[0].runs[0].font.size = Pt(30)

textbox = title_slide.shapes.add_textbox(Inches(1), Inches(1.5), Inches(8), Inches(5))
text_frame = textbox.text_frame
text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER  # Center align all text

# Add Subtitle
title_slide_subtitle = text_frame.add_paragraph()
title_slide_subtitle.text = f"Data Review of Goat Activity \n From: [{start_date.strftime('%m/%d/%Y')}] - [{end_date.strftime('%m/%d/%Y')}]"
title_slide_subtitle.runs[0].font.underline = True
title_slide_subtitle.alignment = PP_ALIGN.CENTER
run = title_slide_subtitle.runs[0]
run.font.size = Pt(24)

# Add space
title_slide_subtitle = text_frame.add_paragraph()
title_slide_subtitle.text = " "
title_slide_subtitle.alignment = PP_ALIGN.CENTER
run = title_slide_subtitle.runs[0]
run.font.size = Pt(18)

# Add Goat Information
title_goats = text_frame.add_paragraph()
title_goats.text = "Goats:"
title_goats.runs[0].font.size = Pt(14)
title_goats.runs[0].font.underline = True
title_goats.alignment = PP_ALIGN.CENTER

# Add space
title_slide_subtitle = text_frame.add_paragraph()
title_slide_subtitle.text = " "
title_slide_subtitle.alignment = PP_ALIGN.CENTER
run = title_slide_subtitle.runs[0]
run.font.size = Pt(18)

goat_info = df_goat_ar['Goat Name'].tolist()

for goat in goat_info:
    p = text_frame.add_paragraph()
    p.text = goat
    p.runs[0].font.size = Pt(14)
    p.alignment = PP_ALIGN.CENTER

# Add Second Slide
second_slide_layout = prs.slide_layouts[5]  # Title Only Layout
second_slide = prs.slides.add_slide(second_slide_layout)
title = second_slide.shapes.title
title.text = "The My Goat Triple Threat"
title.text_frame.paragraphs[0].runs[0].font.bold = True
title.text_frame.paragraphs[0].runs[0].font.size = Pt(25)
title.text_frame.paragraphs[0].space_after = Pt(0.5)

textbox = second_slide.shapes.add_textbox(Inches(1), Inches(1.5), Inches(8), Inches(5))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame

# Add Highlighted Text
title_slide_subtitle = text_frame.add_paragraph()
title_slide_subtitle.text = "Total Cost of Ownership"
title_slide_subtitle.alignment = PP_ALIGN.CENTER
run = title_slide_subtitle.runs[0]
run.font.size = Pt(14)
run.font.bold = True
run.font.color.rgb = RGBColor(0, 128, 0)  # Green Highlight

# Add Cost Savings Content
sections = [
    ('Cost Savings:',
     report_1.replace("Cost Savings:", "").lstrip("\n")),
    ('Labor Multiplier:', report_1_2.replace("Labor Multiplier:", "").lstrip("\n")),
    ('Sustainability/Safety Impact:',report_1_3.replace("Sustainability/Safety Impact:", "").lstrip("\n")),
]

for title, section in sections:
    p = text_frame.add_paragraph()
    title_run = p.add_run()
    title_run.text = title
    p.space_after = Pt(0)
    title_run.font.bold = True
    title_run.font.size = Pt(14)
    p = text_frame.add_paragraph()
    p.text = section
    for run in p.runs:
        run.font.size = Pt(10)
    p.alignment = PP_ALIGN.LEFT


# Add Third Slide
third_slide_layout = prs.slide_layouts[5]  # Title Only Layout
third_slide = prs.slides.add_slide(third_slide_layout)
title = third_slide.shapes.title
title.text = "Total Cost of Ownership"
title.text_frame.paragraphs[0].runs[0].font.bold = True
title.text_frame.paragraphs[0].runs[0].font.size = Pt(40)

textbox = third_slide.shapes.add_textbox(Inches(1), Inches(1.5), Inches(8), Inches(3))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame


# Define the image file path
# chart_path = r"B:\MyGoat\tco_chart.png"

# Add Image to Third Slide
third_slide.shapes.add_picture(chart_path, Inches(1), Inches(1.5), width=Inches(8), height=Inches(4))

# Add TCO Chart Interpretation
textbox = third_slide.shapes.add_textbox(Inches(1), Inches(6), Inches(8), Inches(2))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame

p = text_frame.add_paragraph()
p.text = "TCO Chart & Interpretation"
p.space_after = Pt(2)
p.alignment = PP_ALIGN.LEFT
p.runs[0].font.bold = True
p.runs[0].font.size = Pt(14)

p = text_frame.add_paragraph()
p.text = result_2_1.replace("TCO Chart & Interpretation:", "").lstrip("\n")
p.alignment = PP_ALIGN.LEFT
for run in p.runs:
    run.font.size = Pt(11)


# Add Fourth Slide
fourth_slide_layout = prs.slide_layouts[5]  # Title Only Layout
fourth_slide = prs.slides.add_slide(fourth_slide_layout)
title = fourth_slide.shapes.title
title.text = "Autonomy Rate & Property Appearance Rating"
title.text_frame.paragraphs[0].runs[0].font.bold = True
title.text_frame.paragraphs[0].runs[0].font.size = Pt(30)

# Add Autonomy Rate Chart (Left Side)
# chart_path = r"B:\MyGoat\autonomy_chart.png"
fourth_slide.shapes.add_picture(chart_path_2, Inches(0.5), Inches(1.5), width=Inches(5), height=Inches(4))

# Add KPI Table (Right Side)
table = fourth_slide.shapes.add_table(rows=2, cols=2, left=Inches(5.5), top=Inches(2.3), width=Inches(4), height=Inches(2.5)).table

table.cell(0, 0).text = df_ar.columns[0]
table.cell(0, 1).text = df_ar.columns[1]
table.cell(1, 0).text = df_ar['Original KPI'][0]
table.cell(1, 1).text = df_ar['My Goat Performance'][0]

# Format table headers
table.cell(0, 0).text_frame.paragraphs[0].runs[0].font.bold = True
table.cell(0, 1).text_frame.paragraphs[0].runs[0].font.bold = True

# Highlight Key Performance Message
# Adjust Font Size for Table
for row in range(2):
    for col in range(2):
        for paragraph in table.cell(row, col).text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.size = Pt(12)  # Adjust font size as needed

# Highlight Key Performance Message
table.cell(1, 1).text_frame.paragraphs[0].runs[0].font.bold = True
table.cell(1, 1).text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(0, 0, 0)
table.cell(1, 1).fill.solid()

if "not exceed" in df_ar['My Goat Performance'][0].lower():
    table.cell(1, 1).fill.fore_color.rgb = RGBColor(255, 0, 0)  # Red Highlight
else:
    table.cell(1, 1).fill.fore_color.rgb = RGBColor(0, 255, 0)  # Green Highlight

# table.cell(1, 1).fill.solid()
# table.cell(1, 1).fill.fore_color.rgb = RGBColor(0, 255, 0)  # Green Highlight

# Add Autonomy Ratet Interpretation
textbox = fourth_slide.shapes.add_textbox(Inches(1), Inches(6), Inches(8), Inches(2))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame

p = text_frame.add_paragraph()
p.text = result_3_1
p.alignment = PP_ALIGN.LEFT
for run in p.runs:
    run.font.size = Pt(12)



# Add Fifth Slide
fifth_slide_layout = prs.slide_layouts[5]  # Title Only Layout
fifth_slide = prs.slides.add_slide(fifth_slide_layout)
title = fifth_slide.shapes.title
title.text = "Error Type Distribution"
title.text_frame.paragraphs[0].runs[0].font.bold = True
title.text_frame.paragraphs[0].runs[0].font.size = Pt(30)

textbox = fifth_slide.shapes.add_textbox(Inches(1), Inches(1.5), Inches(8), Inches(3))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame


# Define the image file path
# chart_path_3 = r"B:\MyGoat\error_desc_1.png"

# Add Image to Fifth Slide
fifth_slide.shapes.add_picture(chart_path_3, Inches(1), Inches(1.5), width=Inches(8), height=Inches(4))

# Add Chart 1: Error Type Distribution Interpretation
textbox = fifth_slide.shapes.add_textbox(Inches(1), Inches(6), Inches(8), Inches(2))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame


p = text_frame.add_paragraph()
p.text = "Chart 1: Error Type Distribution"
p.space_after = Pt(2)
p.alignment = PP_ALIGN.LEFT
p.runs[0].font.bold = True
p.runs[0].font.size = Pt(14)

p = text_frame.add_paragraph()
p.text = result_4_1.replace("Chart 1: Error Type Distribution", "").lstrip("\n")
p.alignment = PP_ALIGN.LEFT
for run in p.runs:
    run.font.size = Pt(11)


# Add Sixth Slide
sixth_slide_layout = prs.slide_layouts[5]  # Title Only Layout
sixth_slide = prs.slides.add_slide(sixth_slide_layout)
title = sixth_slide.shapes.title
title.text = "Distribution of Resolution Times"
title.text_frame.paragraphs[0].runs[0].font.bold = True
title.text_frame.paragraphs[0].runs[0].font.size = Pt(30)

# Add Distribution Chart (Left Side)
sixth_slide.shapes.add_picture(chart_path_4, Inches(1), Inches(1.5), width=Inches(8), height=Inches(4))

# Add Response Time Summary Below the Chart
textbox = sixth_slide.shapes.add_textbox(Inches(1), Inches(5), Inches(8), Inches(2))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame

p = text_frame.add_paragraph()
p.text = "Total Average Response Time Per Ticket (Hrs):"
run = p.add_run()
run.text = f''' {Tickets_Data['Duration (Hrs)'].mean():,.2f}'''
p.alignment = PP_ALIGN.LEFT
run.font.bold = True
run.font.size = Pt(18)
run.font.color.rgb = RGBColor(255, 0, 0)
run.underline = True
p.alignment = PP_ALIGN.LEFT
p.runs[0].font.bold = True
p.runs[0].font.size = Pt(14)
p.runs[0].font.color.rgb = RGBColor(0, 0, 0)
p.runs[0].underline = True


textbox = sixth_slide.shapes.add_textbox(Inches(1), Inches(6), Inches(8), Inches(2))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame


p = text_frame.add_paragraph()
p.text = "Chart 2: Average Resolution Time by Goat"
p.space_after = Pt(2)
p.alignment = PP_ALIGN.LEFT
p.runs[0].font.bold = True
p.runs[0].font.size = Pt(14)

p = text_frame.add_paragraph()
p.text = result_4_2.replace("Chart 2: Average Resolution Time by Goat", "").lstrip("\n")
p.alignment = PP_ALIGN.LEFT
for run in p.runs:
    run.font.size = Pt(10)

# Add Seventh Slide
seventh_slide_layout = prs.slide_layouts[5]  # Title Only Layout
seventh_slide = prs.slides.add_slide(seventh_slide_layout)
title = seventh_slide.shapes.title
title.text = "Next Steps & Feedback"
title.text_frame.paragraphs[0].runs[0].font.bold = True
title.text_frame.paragraphs[0].runs[0].font.size = Pt(30)

textbox = seventh_slide.shapes.add_textbox(Inches(1), Inches(1.5), Inches(8), Inches(4))
textbox.text_frame.word_wrap = True
text_frame = textbox.text_frame

p = text_frame.add_paragraph()
p.text = feedback_result
p.alignment = PP_ALIGN.LEFT
for run in p.runs:
    run.font.size = Pt(10)


# Save Presentation
ppt_file = f"/content/drive/Shareddrives/Goat Shed/Goat Database/Automated_Report_{Tickets_Data['Customer'].head(1)[0]}_{datetime.now().strftime('%d-%m-%Y')}.pptx"
prs.save(ppt_file)
print(f"PowerPoint Report saved as {ppt_file}")


PowerPoint Report saved as /content/drive/Shareddrives/Goat Shed/Goat Database/Automated_Report_Greg Cooley Inc_04-03-2025.pptx
